# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import pandas as pd
import numpy as np

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')
# Let's change a little bit the dataframe to make it more easy to work with,
salaries.columns = salaries.columns.str.lower().str.replace(' ','_')

Examine the `salaries` dataset using the `head` function below.

In [4]:
# Your code here:
salaries.head()

,name,job_titles,department,full_or_part-time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:
salaries.isna().sum()

name                     0
job_titles               0
department               0
full_or_part-time        0
salary_or_hourly         0
typical_hours        25161
annual_salary         8022
hourly_rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
# Your code here:
a = [i for i in salaries.salary_or_hourly.value_counts()]
print(a[0]/a[1])
print(salaries.salary_or_hourly.value_counts())
# Notice that typical_hours and hourly_rate have as many elemnts as salary and likewise annual_salary has as many elements Hourly

3.1364996260284217
Salary    25161
Hourly     8022
Name: salary_or_hourly, dtype: int64


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [7]:
# Your code here:
salaries.department.value_counts()[:2]

POLICE    13414
FIRE       4641
Name: department, dtype: int64

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [8]:
# Your code here:
from statsmodels.stats import weightstats as w
sample = salaries[salaries.salary_or_hourly == 'Hourly']
stats, pval =w.ztest(sample.hourly_rate, value=30)
print(stats, pval)
# With a certanity of over a 99.999999% we can affirm that the hourly wage of hourly workers is significantly different from 30$/hr, 
# actually we observe that is a little greater

20.6198057854942 1.8228873859286195e-94


/home/pomilusky/anaconda3/envs/ironhack/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [9]:
# Your code here:
sample = salaries[(salaries.department == 'POLICE')&(salaries.salary_or_hourly == 'Salary')]
stats, pval =w.ztest(sample.annual_salary, value=86000)
print(f"With a certany of a {100-100*pval:.2f} % we can affirm that the chief of Police was telling the truth.")

With a certany of a 99.79 % we can affirm that the chief of Police was telling the truth.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [10]:
# Your code here:
cros = pd.crosstab(salaries.department, salaries.salary_or_hourly)
cros[cros.Hourly == max(cros.Hourly)]


salary_or_hourly,Hourly,Salary
department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [11]:
# Your code here:
sample = salaries[(salaries.salary_or_hourly == 'Hourly')&(salaries.department == 'STREETS & SAN')]
stats, pval =w.ztest(sample.hourly_rate, value=35)
print(stats, pval)
print(f"They are, most likely, telling the truth. with a confidence level of {100-100*pval:.2f} % their salary is under 35$ the hour.")

-9.567447887848152 1.0957711283886869e-21
They are, most likely, telling the truth. with a confidence level of 100.00 % their salary is under 35$ the hour.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [12]:
# Your code here:
from scipy import stats

alpha = 0.95
sample = salaries[(salaries.salary_or_hourly == 'Hourly')]
mu = sample.hourly_rate.mean()
stats.t.interval(alpha, len(sample.hourly_rate)-1, loc=mu, scale=stats.sem(sample.hourly_rate))

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [13]:
# Your code here:
# Your code here:
from scipy import stats

alpha = 0.95
sample = salaries[(salaries.salary_or_hourly == 'Salary')&(salaries.department == 'POLICE')]
mu = sample.annual_salary.mean()
stats.t.interval(alpha, len(sample.annual_salary)-1, loc=mu, scale=stats.sem(sample.annual_salary))

(86177.05631531784, 86795.77269094894)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [14]:
# Your code here:
from statsmodels.stats.proportion import proportions_ztest as pzt 
stats, pval = pzt(salaries.salary_or_hourly.value_counts()[1],len(salaries),0.25)
print(f" The hourly employed people in the Chicago department is under a 25% of the total with a certany of about {100-100*pval:.2f}.")

 The hourly employed people in the Chicago department is under a 25% of the total with a certany of about 99.96.
